In [ ]:
%load_ext autoreload
%autoreload 2
# default_exp plugin.authenticators.password

In [ ]:
# export 
# hide

import abc
from time import sleep
from pathlib import Path

import pymemri
from pymemri.data.basic import read_file
from pymemri.cvu.utils import get_default_cvu
from pymemri.plugin.states import RUN_USER_ACTION_NEEDED, RUN_USER_ACTION_COMPLETED
from pymemri.data.schema import CVUStoredDefinition
import time

In [ ]:
# export
# hide

class PasswordAuthenticator:
    DEFAULT_CVU = "password_auth.cvu"
    MAX_LOGIN_ATTEMPTS = 3
    SLEEP_INTERVAL = 1.0
    MAX_POLL_TIME = 600

    def __init__(self, client, pluginRun):
        self.client = client
        self.pluginRun = pluginRun
        self.isTest = False

    def authenticate(self, login_callback):
        self.request_user_credentials()

        login_success = False
        for i in range(self.MAX_LOGIN_ATTEMPTS):
            username, password = self.poll_credentials()
            try:
                login_callback(username, password)
                login_success = True
                break
            except Exception as e:
                print("Login failed, invalid credentials.")
                if self.pluginRun.account:
                    attempts_remaining = self.MAX_LOGIN_ATTEMPTS - (i + 1)
                    account = self.pluginRun.account[0]
                    account.errorMessage = f"Reached max login attempts. {attempts_remaining} attempts remaining"
                    self.client.update_item(account)
                
        if not login_success:
            self.pluginRun.status = "error"
            self.client.update_item(self.pluginRun)
            
            if self.pluginRun.account:
                account = self.pluginRun.account[0]
                account.errorMessage = "Reached max login attempts."
                self.client.update_item(account)

            raise RuntimeError("Reached max login attempts.")
            
    def request_user_credentials(self):
        cvu = get_default_cvu(self.DEFAULT_CVU)
        self.client.create(cvu)
        self.pluginRun.add_edge("view", cvu)
        self.client.create_edge(self.pluginRun.get_edges("view")[0])
        self.pluginRun.status = RUN_USER_ACTION_NEEDED
        self.client.update_item(self.pluginRun)

    def poll_credentials(self):
        start_time = time.time()
        while True:
            if time.time() - start_time > self.MAX_POLL_TIME:
                self.pluginRun.status = "error"
                self.client.update_item(self.pluginRun)
                raise RuntimeError("Stop polling, max time reached.")
                
            print("polling for credentials...")
            sleep(self.SLEEP_INTERVAL)
            self.pluginRun = self.client.get(self.pluginRun.id)
            if self.pluginRun.status == RUN_USER_ACTION_COMPLETED:
                account = self.pluginRun.account[0]
                return account.identifier, account.secret

In [ ]:
from pymemri.plugin.pluginbase import PluginBase
from pymemri.plugin.schema import PluginRun, Account
from pymemri.pod.client import PodClient
import threading

class MyAuthenticatedPlugin(PluginBase):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.logged_in = False
        self.authenticator = PasswordAuthenticator(kwargs["client"], kwargs["pluginRun"])
        
    def login(self, username, password):
        if not (username=="username" and password=="password"):
            raise ValueError("Wrong credentials.")
            
    def run(self):
        self.authenticator.authenticate(login_callback=self.login)
        self.logged_in = True
        print("done!")
    
    def add_to_schema(self):
        pass

pod_client = PodClient()

run = PluginRun("", "", "")
account = Account(service="myAccount")
run.add_edge("account", account)
run.status = "started"

pod_client.create(run)
pod_client.create(account)
pod_client.create_edge(run.get_edges("account")[0])

True

In [ ]:
# Create Plugin
plugin = MyAuthenticatedPlugin(client=pod_client, pluginRun=run)

# Start plugin in background thread
def run_plugin():
    plugin.run()
    assert plugin.logged_in
    
thread = threading.Thread(target=run_plugin)
thread.start()

In [ ]:
# Enter password and check if plugin is authenticated

def simulate_enter_password(pod_client, run_id):
    run = pod_client.get(run_id)
    account = run.account[0]

    username = "username"
    password = "password"
    account.identifier = username
    account.secret = password
    run.status = "ready"

    pod_client.update_item(account)
    pod_client.update_item(run)

time.sleep(2)
simulate_enter_password(pod_client, run.id)
time.sleep(2)

thread.join()
assert plugin.logged_in

polling for credentials...
polling for credentials...
done!


In [ ]:
# hide
from nbdev.export import notebook2script
notebook2script()

Converted basic.ipynb.
Converted cvu.utils.ipynb.
Converted data.photo.ipynb.
Converted importers.Importer.ipynb.
Converted importers.util.ipynb.
Converted index.ipynb.
Converted indexers.indexer.ipynb.
Converted itembase.ipynb.
Converted plugin.authenticators.password.ipynb.
Converted plugin.pluginbase.ipynb.
Converted plugin.schema.ipynb.
Converted plugin.stateful.ipynb.
Converted plugin.states.ipynb.
Converted pod.client.ipynb.
Converted pod.db.ipynb.
Converted pod.utils.ipynb.
Converted test_utils.ipynb.
